In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.style import use

use('fivethirtyeight')

In [ ]:
# Load dataset
exchange_rates = pd.read_csv('euro-daily-hist_1999_2020.csv')

In [ ]:
# First 5 rows
exchange_rates.head()

In [ ]:
# Last 5 rows
exchange_rates.tail()

In [ ]:
# Show dataset info
exchange_rates.info()

In [ ]:
# Null values
exchange_rates.isnull().sum()

In [ ]:
# Rename columns to more readable format - remove special characters and spaces
exchange_rates.rename(columns={'[US dollar ]': 'US_dollar',
                               'Period\\Unit:': 'Time'},
                      inplace=True)

# Convert Time column to datetime format for proper date handling
exchange_rates['Time'] = pd.to_datetime(exchange_rates['Time'])

# Sort the dataframe by Time column in ascending order
exchange_rates.sort_values('Time', inplace=True)

# Reset index after sorting to maintain clean sequential numbering
exchange_rates.reset_index(drop=True, inplace=True)

In [ ]:
# Create a new DataFrame containing only the Time and US_dollar columns from exchange_rates
euro_to_dollar = exchange_rates[['Time', 'US_dollar']]

In [ ]:
euro_to_dollar['US_dollar'].value_counts()

In [ ]:
# Drop rows where '-' appears in US_dollar column
euro_to_dollar = euro_to_dollar[euro_to_dollar['US_dollar'] != '-']
euro_to_dollar.head()

In [ ]:
# Change US_dollar column to float
euro_to_dollar['US_dollar'] = pd.to_numeric(euro_to_dollar['US_dollar'])
euro_to_dollar['US_dollar'].dtype

In [ ]:
# Plot Time and US_dollar column
plt.plot(euro_to_dollar['Time'], euro_to_dollar['US_dollar'])
plt.show()

In [ ]:
# Calculate rolling mean for 30 days and assign to rolling_mean column
euro_to_dollar['rolling_mean'] = euro_to_dollar['US_dollar'].rolling(30).mean()

In [ ]:
euro_to_dollar

In [ ]:
# Plot rolling mean and time
euro_to_dollar.plot(x='Time', y='rolling_mean')
plt.show()

In [ ]:
# Filter data to separate dataframes for respective timeframe (covid, pre-covid)
euro_to_dollar_covid = euro_to_dollar[euro_to_dollar['Time'] >= '2020-01-01']
euro_to_dollar_pre_covid = euro_to_dollar[~(euro_to_dollar['Time'] >= '2020-01-01') & (euro_to_dollar['Time'] >= '2016-01-01')]

In [ ]:
# Calculate percent gain from first entry in euro_to_dollar_covid to last entry
percent_gain_covid = ((euro_to_dollar_covid['US_dollar'].iloc[-1] - euro_to_dollar_covid['US_dollar'].iloc[0]) / euro_to_dollar_covid['US_dollar'].iloc[0]) * 100

# Get max USD value within covid dataframe
max_covid = euro_to_dollar_covid['US_dollar'].max()

In [ ]:
# Print percent gain
print(percent_gain_covid)

In [ ]:
# Create a figure with one subplot and set the size
fig, ax = plt.subplots(1, 1, figsize=(15, 5))

# Remove tick marks from all sides of the plot
ax.tick_params(bottom=False, top=False, left=False, right=False)

# Remove the border (spines) from all sides of the plot
for location in ['top', 'bottom', 'left', 'right']:
    ax.spines[location].set_visible(False)

# Plot pre-COVID data with reduced opacity
ax.plot(euro_to_dollar_pre_covid['Time'], euro_to_dollar_pre_covid['rolling_mean'], alpha=0.5, c='#af0b1e')

# Plot COVID period data with full opacity
ax.plot(euro_to_dollar_covid['Time'], euro_to_dollar_covid['rolling_mean'], c='#af0b1e')

# Get the initial date and exchange rate value for COVID period
x_pos = euro_to_dollar_covid['Time'].iloc[0]  # First date
y_pos = euro_to_dollar_covid['rolling_mean'].iloc[0]  # Corresponding y-value

# Add grey shaded region to highlight COVID period
ax.axvspan(xmin=18262, xmax=18630, ymin=0.09, alpha=0.3, color='grey')

# Add main title and subtitle to the plot
ax.text(16600, y_pos + 0.165, 'Euro-USD rate increased 9.4% during the COVID-19 pandemic', size=24, weight='bold')
ax.text(16605, y_pos + 0.15, 'Euro-USD Exchange Rates Between 2016 and 2021', size=16)

# Add footer with copyright and source information
ax.text(
    16600.0,
    1.015,
    '©JASTRO-DEV' + ' ' * 238 + 'Source: European Central Bank',
    color='#f0f0f0',
    backgroundcolor='#4d4d4d',
    size=10,
)

# Display the plot
plt.savefig('covid_exchange_rate_line_plot.png', bbox_inches='tight', dpi=200, pad_inches=0.1)
plt.show()